In [1]:
# Necessary imports
from langchain_openai import ChatOpenAI
from langchain.agents import tool, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

In [2]:
# Load LLM

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [7]:
# Define some tools

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers together."""
    return a * b

@tool
def subtract(a: int, b: int):
    """Subtracts b from a."""
    return a-b

tools = [add, subtract]

In [8]:
# Create prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly and helpful math tutor for kindergarten students"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [9]:
# Bind the tools to the LLM

llm_with_tools = llm.bind_tools(tools)

In [12]:
# Create agent

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
list(agent_executor.stream({"input": "what is five minus three"}))



> Entering new AgentExecutor chain...

Invoking: `subtract` with `{'a': 5, 'b': 3}`


2Five minus three is equal to two.

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='subtract', tool_input={'a': 5, 'b': 3}, log="\nInvoking: `subtract` with `{'a': 5, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_AU2sa2xeK3Bn1ZlGG1QbB2fx', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'subtract'}, 'type': 'function'}]})], tool_call_id='call_AU2sa2xeK3Bn1ZlGG1QbB2fx')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_AU2sa2xeK3Bn1ZlGG1QbB2fx', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'subtract'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='subtract', tool_input={'a': 5, 'b': 3}, log="\nInvoking: `subtract` with `{'a': 5, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_AU2sa2xeK3Bn1ZlGG1QbB2fx', 'function': {'arguments': '{\n  "a": 5,\n  "b": 3\n}', 'name': 'subtra

In [15]:
# now adding memory to the ai model. WE must add a place for memory variables and then keep track of the history.

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly and helpful math tutor for kindergarten students"),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [23]:
#set up a list to track chat history

chat_history=[]

In [24]:
#Remake the agent_executor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [25]:
#now we have to track the inputs and outputs as chat history

input1 = "what is twenty three plus eighty six"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

input2 = "what is that minus six"
result2 = agent_executor.invoke({"input": input2, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input2),
        AIMessage(content=result2["output"]),
    ]
)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 23, 'b': 86}`


1978Twenty three plus eighty six is equal to one hundred and nine.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `subtract` with `{'a': 109, 'b': 6}`


103One hundred and nine minus six is equal to one hundred and three.

> Finished chain.


In [26]:
chat_history

[HumanMessage(content='what is twenty three plus eighty six'),
 AIMessage(content='Twenty three plus eighty six is equal to one hundred and nine.'),
 HumanMessage(content='what is that minus six'),
 AIMessage(content='One hundred and nine minus six is equal to one hundred and three.')]